In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import coint, grangercausalitytests, adfuller
from scipy.stats import pearsonr
import pytz
import requests

import warnings
warnings.filterwarnings('ignore')

In [ ]:
API_KEY_ID     = "PKL9BT6QPZNC5690ELR5"
API_SECRET_KEY = "8N12VHY2aZ4qbMHar3gvWdDJpaws5fRZMyQZO3Bf"


# Step 1: Data Retrieval using Alpaca API
def fetch_alpaca_data(symbol, timeframe, start_date, end_date='2025-03-17'):
    url = 'https://data.alpaca.markets/v2/stocks/bars'
    headers = {
        'APCA-API-KEY-ID': API_KEY_ID,
        'APCA-API-SECRET-KEY': API_SECRET_KEY
    }

    params = {
        'symbols': symbol,
        'timeframe': timeframe,
        'start': dt.datetime.strptime(start_date, "%Y-%m-%d").isoformat() + 'Z',
        'end': dt.datetime.strptime(end_date, "%Y-%m-%d").isoformat() + 'Z',
        'limit': 10000,
        'adjustment': 'raw',
        'feed': 'sip'
    }

    data_list = []
    eastern = pytz.timezone('America/New_York')
    utc = pytz.utc

    print("Starting data fetch...")
    while True:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"Error fetching data with status code {response.status_code}: {response.text}")
            break

        data = response.json()
        bars = data.get('bars')

        for symbol, entries in bars.items():
            for entry in entries:
                try:
                    utc_time = dt.datetime.fromisoformat(entry['t'].rstrip('Z')).replace(tzinfo=utc)
                    eastern_time = utc_time.astimezone(eastern)
                    data_entry = {
                        'DATE': eastern_time,
                        f'{symbol}': entry['c']
                    }
                    data_list.append(data_entry)
                except Exception as e:
                    continue

        if 'next_page_token' in data and data['next_page_token']:
            params['page_token'] = data['next_page_token']
        else:
            break

    df = pd.DataFrame(data_list)
    print("Data fetching complete.")
    return df

# Step 2: Lead-Lag Analysis using Cointegration
def cointegration_test(series1, series2):
    coint_score, p_value, _ = coint(series1, series2)
    return coint_score, p_value

# Step 3: Lead-Lag Analysis using EWMA
def ewma_lead_lag(series1, series2, span=60):
    ewma1 = series1.ewm(span=span).mean()
    ewma2 = series2.ewm(span=span).mean()
    correlation, _ = pearsonr(ewma1.dropna(), ewma2.dropna())
    return ewma1, ewma2, correlation

# Step 4: Granger Causality for Optimal Lag
def granger_causality(series1, series2, maxlag=30):
    data = pd.concat([series1, series2], axis=1).dropna()
    result = grangercausalitytests(data, maxlag=maxlag, verbose=False)
    p_values = [round(result[i+1][0]['ssr_ftest'][1], 4) for i in range(maxlag)]
    optimal_lag = p_values.index(min(p_values)) + 1
    return optimal_lag, p_values

# Main Execution
stock1_data = fetch_alpaca_data('NVDA', '1Min', '2025-01-01')
stock2_data = fetch_alpaca_data('TSLA', '1Min', '2025-01-01')

# Synchronize the data on dates
merged_data = pd.merge(stock1_data, stock2_data, on='DATE', how='inner')

# Calculate Cointegration
score, p_value = cointegration_test(merged_data['NVDA'], merged_data['TSLA'])
print(f'Cointegration Test: Score={score:.4f}, P-Value={p_value:.4f}')

# Calculate EWMA
ewma1, ewma2, correlation = ewma_lead_lag(merged_data['NVDA'], merged_data['TSLA'])
print(f'EWMA Correlation: {correlation:.4f}')

# Optimal Lag via Granger Causality
lag, p_values = granger_causality(merged_data['NVDA'], merged_data['TSLA'])
print(f'Optimal Lag: {lag} | P-Values: {p_values}')

# Plotting
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.title('Intraday Prices')
plt.plot(merged_data['DATE'], merged_data['NVDA'], label='NVDA')
plt.plot(merged_data['DATE'], merged_data['TSLA'], label='TSLA', alpha=0.7)
plt.legend()

plt.subplot(2, 1, 2)
plt.title('EWMA of Returns')
plt.plot(ewma1, label='NVDA EWMA')
plt.plot(ewma2, label='TSLA EWMA', alpha=0.7)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:

# Simulating merged_data for demonstration purposes
np.random.seed(42)
dates = pd.date_range(start="2025-01-01", periods=100, freq="T")
spy_prices = np.cumsum(np.random.randn(100)) + 100
qqq_prices = np.cumsum(np.random.randn(100)) + 200
merged_data = pd.DataFrame({"DATE": dates, "SPY": spy_prices, "QQQ": qqq_prices})

# Define a function to calculate EWMA with a given lag
def calculate_ewma(data, lag):
    alpha = 2 / (lag + 1)  # Smoothing factor based on lag
    return data.ewm(alpha=alpha, adjust=False).mean()

# Compute EWMA for a range of lags and store results
lags = range(1, 11)
ewma_results = {}
for lag in lags:
    ewma_results[lag] = {
        'EWMA_SPY': calculate_ewma(merged_data['SPY'], lag),
        'EWMA_QQQ': calculate_ewma(merged_data['QQQ'], lag)
    }

# Add EWMA with an example lag (e.g., lag=5) to the dataframe
example_lag = 5
merged_data['EWMA_SPY'] = ewma_results[example_lag]['EWMA_SPY']
merged_data['EWMA_QQQ'] = ewma_results[example_lag]['EWMA_QQQ']

# Perform linear regression on SPY and QQQ prices
X = merged_data['SPY'].values.reshape(-1, 1)  # Independent variable
Y = merged_data['QQQ'].values  # Dependent variable

model = LinearRegression()
model.fit(X, Y)

# Get regression coefficients
slope = model.coef_[0]
intercept = model.intercept_

# Predict QQQ prices using the regression model
merged_data['QQQ_Predicted'] = model.predict(X)

# Create lagged features for SPY and QQQ prices
max_lag = 10
for lag in range(1, max_lag + 1):
    merged_data[f'SPY_Lag_{lag}'] = merged_data['SPY'].shift(lag)
    merged_data[f'QQQ_Lag_{lag}'] = merged_data['QQQ'].shift(lag)

# Drop rows with NaN values due to lagging
merged_data.dropna(inplace=True)

# Train a Random Forest model using lagged features
features = [col for col in merged_data.columns if 'Lag' in col]
X_rf = merged_data[features]
Y_rf = merged_data['QQQ']

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_rf, Y_rf)

# Plotting EWMA for all lags
plt.figure(figsize=(12, 6))
for lag in lags:
    plt.plot(ewma_results[lag]['EWMA_SPY'], label=f'SPY EWMA Lag {lag}')
plt.legend()
plt.title('EWMA for SPY with Different Lags')
plt.show()

plt.figure(figsize=(12, 6))
for lag in lags:
    plt.plot(ewma_results[lag]['EWMA_QQQ'], label=f'QQQ EWMA Lag {lag}')
plt.legend()
plt.title('EWMA for QQQ with Different Lags')
plt.show()

# Output results: Regression coefficients and Random Forest feature importance
print(f"Linear Regression Slope: {slope}")
print(f"Linear Regression Intercept: {intercept}")
print("Random Forest Feature Importance:")
print(rf_model.feature_importances_)


In [ ]:

# Fetch data for SPY and QQQ
spy_data = fetch_alpaca_data('SPY', '1Min', '2025-01-01')
qqq_data = fetch_alpaca_data('QQQ', '1Min', '2025-01-01')

# Merge data on date
merged_data = pd.merge(spy_data, qqq_data, on='DATE', how='inner')

# Perform cointegration test
score, pvalue, _ = coint(merged_data['SPY'], merged_data['QQQ'])
print(f"Cointegration Test: p-value = {pvalue}")

# If p-value < 0.05, the series are likely cointegrated
if pvalue < 0.05:
    print("Series are cointegrated.")
else:
    print("Series are not cointegrated.")

# Calculate EWMA
alpha = 0.1  # Smoothing factor
ewma_spy = merged_data['SPY'].ewm(alpha=alpha, adjust=False).mean()
ewma_qqq = merged_data['QQQ'].ewm(alpha=alpha, adjust=False).mean()

# Plot EWMA
plt.figure(figsize=(12,6))
plt.plot(ewma_spy, label='SPY EWMA')
plt.plot(ewma_qqq, label='QQQ EWMA')
plt.legend()
plt.title('EWMA Comparison')
plt.show()

# Perform Granger causality test to check for lead-lag relationship
test_result = grangercausalitytests(merged_data[['SPY', 'QQQ']], maxlag=10, verbose=False)
for lag, result in test_result.items():
    print(f"Lag: {lag}, p-value: {result[0]['ssr_chi2test'][1]}")

# Prepare data for regression
X = merged_data[['SPY']]
y = merged_data['QQQ']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Random Forest
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

# Determine optimal lag using Granger causality results
optimal_lag = min([lag for lag, result in test_result.items() if result[0]['ssr_chi2test'][1] < 0.05], default=None)
if optimal_lag:
    print(f"Optimal lag based on Granger causality: {optimal_lag}")
else:
    print("No significant lag found.")

# Plot original data with lag
if optimal_lag:
    plt.figure(figsize=(12,6))
    plt.plot(merged_data['SPY'], label='SPY')
    plt.plot(merged_data['QQQ'].shift(optimal_lag), label=f'QQQ shifted by {optimal_lag}')
    plt.legend()
    plt.title('Lead-Lag Relationship')
    plt.show()
